In [38]:
import requests
from datetime import datetime
import telebot
from telebot import types
import sqlite3
import random

import numpy as np
import pandas as pd
import psutil
from IPython.display import Image

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go

import kaleido
import json
import matplotlib.pyplot as plt
import seaborn as sns
import plotly 
import plotly.graph_objects as go
import collections
import plotly.express as px
import plotly.io as pio

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

import statistics

In [39]:
import telebot
from telebot import types
import time
coun=0 #fortinder
from requests import get
starlist=['lll', 'iii', 'sss', 'ttt']

In [40]:
pio.kaleido.scope.plotlyjs = r'D:\PythonModules\plotly\package_data\plotly.min.js'

Это главное меню, тут все норм работает

In [41]:
bot = telebot.TeleBot("1874823468:AAEbRri8tSAL0gIx4DlTXpjHIhWP-0JNmUw")
@bot.message_handler(commands=['start'])
def send_keyboard(message, text="Привет, чем я могу тебе помочь?"):
    keyboard = types.ReplyKeyboardMarkup(row_width=2)  # наша клавиатура
    itembtn1 = types.KeyboardButton('Подбор вакансии') # создадим кнопку
    itembtn2 = types.KeyboardButton('Аналитика')
    itembtn3 = types.KeyboardButton('Избранное')
    itembtn4 = types.KeyboardButton('Мой профиль')
    keyboard.add(itembtn1, itembtn2) 
    keyboard.add(itembtn3, itembtn4) 
    resp = bot.send_message(message.from_user.id,
                     text=text, reply_markup=keyboard)

    bot.register_next_step_handler(resp, hub1)
    


Это навигатор по менюшке

In [42]:
key_word = ''

def hub1(resp): #navigator po menu
    global key_word
    if resp.text == 'Подбор вакансии':
        msg = bot.send_message(resp.from_user.id,
                     text='Укажи название вакансии')
        bot.register_next_step_handler(msg, tndr0)
    elif resp.text == 'Аналитика':
        msg = bot.send_message(resp.from_user.id,
                     text='Укажи название вакансии чтобы получить подробную аналитику')
        bot.register_next_step_handler(msg, anal)
    elif resp.text == 'Избранное':
        msg = bot.send_message(resp.from_user.id,
                     text = 'Вот твое избранное:')
        star(resp)
    elif resp.text == 'Мой профиль':
        bot.send_message(resp.from_user.id,
                     text='Вот твой профиль')
        profile(resp)
    else:
        bot.send_message(resp.from_user.id,
                     text='Некорректно')
        send_keyboard(resp)


## Тиндер
Работающий, вместо джангира просто вакансии прифигачить и в да функцию сохранения в переменную старлист

In [52]:
def tndr0(resp):
    global key_word
    key_word = resp.text
    tndr(resp)

def tndr(resp): # tinder
    global coun
    coun+=1
    keyboard = types.ReplyKeyboardMarkup(row_width=2)  # наша клавиатура
    itembtn1 = types.KeyboardButton('Да') # создадим кнопку
    itembtn2 = types.KeyboardButton('Нет')
    itembtn3 = types.KeyboardButton('Астанавитес')
    keyboard.add(itembtn1, itembtn2, itembtn3) 
    #начало говнокода
    vacancy_info = get_vacancy_main(key_word, coun)
    try:
        vacancy_info = f"{vacancy_info[0]}\n{vacancy_info[1]}\n{vacancy_info[2]}\n{vacancy_info[3]}"
        msg=bot.send_message(resp.from_user.id,
                        text=vacancy_info, reply_markup=keyboard)
    except:
        msg=bot.send_message(resp.from_user.id, text = 'По этому запросу вакансий нет или они закончились. Выйдите из этого раздела', reply_markup=keyboard)
        #bot.register_next_step_handler(msg, send_keyboard)
    #конец говнокода
    bot.register_next_step_handler(msg, tndr2)

def tndr2(resp):
    global coun
    if resp.text == 'Да':
        bot.send_message(resp.from_user.id, text='saved')
        tndr(resp)
    elif resp.text == 'Астанавитес':
        message = bot.send_message(resp.from_user.id,text='Пиздуй-бороздуй')
        coun = 0
        send_keyboard(resp)
    else:
        bot.send_message(resp.from_user.id, text='Приму это за нет и сохранять не буду')
        tndr(resp)



## Для аналитики
 вместо пикчи график, вместо текста чет полезное, прост подставить

In [44]:
def anal(resp):
    keyboard = types.ReplyKeyboardMarkup(row_width=2) 
    itembtn1 = types.KeyboardButton('Ищо') # создадим кнопку
    itembtn2 = types.KeyboardButton('Не ищо')
    keyboard.add(itembtn1, itembtn2) 
    msg = bot.send_message(resp.from_user.id, text='Здесь появится графичек. Возможно придется подождать',reply_markup=keyboard)
    #начало говнокода
    data = all_salaries(resp.text)[0]
    try: 
        result_photo = histogram(data).to_image(format="png", engine="kaleido")
        bot.send_photo(resp.chat.id, result_photo)
        bot.send_message(resp.chat.id, f'максимум: {max(data)}')
        bot.send_message(resp.chat.id, f'минимум: {min(data)}')
        bot.send_message(resp.chat.id, f'средняя зарплата: {int(sum(data)/len(data))}')
        bot.send_message(resp.chat.id, f'медианная зарплата: {statistics.median(data)}')
    except:
        bot.send_message(resp.chat.id, 'введите другой запрос')
    #конец говнокода
    keyboard = types.ReplyKeyboardMarkup(row_width=2)  # наша клавиатура
    bot.register_next_step_handler(msg, anal2)

def anal2(resp):
    if resp.text == 'Ищо':
        msg=bot.send_message(resp.from_user.id, text='Вводи тогда еще одну вакансию')
        bot.register_next_step_handler(msg, anal)
    elif resp.text =='Не ищо':
        msg=bot.send_message(resp.from_user.id, text='Пиздуй-бороздуй')
        send_keyboard(resp)
    else:
        msg=bot.send_message(resp.from_user.id, text='Ты бы лучше на кнопку нажал')
        bot.register_next_step_handler(msg, anal2)

    


## Избранное
старлист - список избранного(настоящий), просто надо его генерить из тиндера. Сюда даже подставлять ниче не надо

In [45]:
def star(resp):
    global starlist
    if starlist!=[]:
        for i in range(len(starlist)):
            msg=bot.send_message(resp.from_user.id, text=str(i+1)+' '+starlist[i])
        keyboard = types.ReplyKeyboardMarkup(row_width=2) 
        itembtn1 = types.KeyboardButton('Удалить 1') # создадим кнопку
        itembtn2 = types.KeyboardButton('Удалить 2')
        itembtn3 = types.KeyboardButton('Удалить все') # создадим кнопку
        itembtn4 = types.KeyboardButton('Назад')
        keyboard.add(itembtn1, itembtn2) 
        keyboard.add(itembtn3, itembtn4) 
        msg=bot.send_message(resp.from_user.id, text='Можешь нажать на кнопку или написать Удалить N чтобы удалить N-ное избранное', 
                             reply_markup=keyboard)
        bot.register_next_step_handler(msg, star2)
    else:
        bot.send_message(resp.from_user.id, text='Нету такого')
        send_keyboard(resp)
    

    
    
def star2(resp):
    global starlist
    if resp.text.split(' ')[0] =='Удалить':
        if resp.text.split(' ')[1].isdigit():
            n=int(resp.text.split(' ')[1])
            try:
                starlist.pop(n-1)
                msg = bot.send_message(resp.from_user.id, 
                                       text='Удалил избранное под выбранным номером. Нажми на кнопку(любую) еще раз')
                bot.register_next_step_handler(msg, star2)
            except:
                msg = bot.send_message(resp.from_user.id, 
                                       text='Число кривое ацтань, нажми на кнопку(любую) еще раз')
                bot.register_next_step_handler(msg, star2)
        if resp.text.split(' ')[1] == 'все':
            starlist = []
            msg = bot.send_message(resp.from_user.id, text='Удалил все')
            send_keyboard(resp)
    if resp.text=='Назад':
        send_keyboard(resp)


## Заполнение профиля
Тут внутрь функций город зп станция подставить работу с полученным респонсом(запись в базу), вместо первого сообщения реальный профиль, все остальное норм

In [46]:
def profile(resp):
    bot.send_message(resp.from_user.id, text='Вотут все данные лежать должны')
    keyboard = types.ReplyKeyboardMarkup(row_width=2) 
    itembtn1 = types.KeyboardButton('Город') # создадим кнопку
    itembtn2 = types.KeyboardButton('Станция метро')
    itembtn3 = types.KeyboardButton('Минимальная зп') # создадим кнопку
    itembtn4 = types.KeyboardButton('Назад')
    keyboard.add(itembtn1, itembtn2) 
    keyboard.add(itembtn3, itembtn4) 
    msg = bot.send_message(resp.from_user.id, text='Выбери раздел в который хочешь внести изменения', 
                             reply_markup=keyboard)
    bot.register_next_step_handler(msg, profile0)
def profile0(resp):
    if resp.text == 'Город':
        msg = bot.send_message(resp.from_user.id, text='Введи изменения')
        bot.register_next_step_handler(msg, city)
    elif resp.text == 'Станция метро':
        msg = bot.send_message(resp.from_user.id, text='Введи изменения')
        bot.register_next_step_handler(msg, metro)
    elif resp.text == 'Минимальная зп':
        msg = bot.send_message(resp.from_user.id, text='Введи изменения')
        bot.register_next_step_handler(msg, payment)
    elif resp.text == 'Назад':
        send_keyboard(resp)
    else:
        msg=bot.send_message(resp.from_user.id, text='Ты бы лучше на кнопку нажал')
        bot.register_next_step_handler(msg, profile0)
def city(resp):
    bot.send_message(resp.from_user.id, text='Изменения приняты, вот ваш обновленный профиль:')
    profile(resp)
def metro(resp):
    bot.send_message(resp.from_user.id, text='Изменения приняты, вот ваш обновленный профиль:')
    profile(resp)
def payment(resp):
    bot.send_message(resp.from_user.id, text='Изменения приняты, вот ваш обновленный профиль:')
    profile(resp)




### Запуск бота

In [51]:
bot.polling(none_stop=True)

# НЕ БОТ (получение данных)

In [47]:
#вспомогательная функция для форматирования инфы о зарплате

def get_salary(sal):
    if sal['currency'] == None:
        sal['currency'] = ''
    if sal['from'] == None and sal['to'] == None:
        t = 'зарплата не указана'
        return t
    if sal['from'] == None:
        t = f'зарплата до {sal["to"]} {sal["currency"]}'
        return t
    if sal['to'] == None:
        t = f'зарплата от {sal["from"]} {sal["currency"]}'
        return t
    else:
        t = f'зарплата от {sal["from"]} {sal["currency"]} до {sal["to"]} {sal["currency"]}'
        return t

In [48]:
# основная функция получения инфы о вакансиях. Принимает запрос, по которому искать вакансию (любой текст)
# и номер вакансии в выдаче (ноль из ста по умолчанию). Если номер указан больше, чем есть в выдаче, то берет последнюю
# Возвращает инфу о названии вакансии, зарплате, регионе (но не городе подробно. Это можно будет прикрутить, если нужно)
# а также ссылку на вакансию с сайта хэ хэ

def get_vacancy_main(search, number = 0):
    data = requests.get(f'https://api.hh.ru/vacancies?text={search}&per_page=100', 
                        headers={"User-Agent":"HH-User-Agent"}).json()
    
    if len(data['items']) == 0:
        return None
    if number > len(data['items']):
        return None
    
    #получаем нужную вакансию из списка
    data = data['items'][min(number, len(data['items']) - 1)]
    
    name = data['name']
    
    try:
        salary = get_salary(data['salary'])
    except:
        salary = 'зарплата не указана'
    
    try:
        area = data['area']['name']
    except:
        area = 'регион не указан'
    
    if data['alternate_url'] != None:
        url = data['alternate_url']
    else:
        url = 'ссылка на вакансию отсутствует'
    
    
    
    return (name, salary, area, url)

In [49]:
def all_salaries(search):
    
    #заводим пустой список для всех зарплат и делаем тестовый запрос, чтобы узнать, сколько страниц в выдаче
    salaries = []
    test = requests.get(f'https://api.hh.ru/vacancies?text={search}&per_page=100',
                        headers={"User-Agent":"HH-User-Agent"}).json()
    pages = test['pages']
    
    #проходимся по всем страницам в выдаче и склеиваем инфу о зарплатах. Парсится максимум 10 страниц, так как иначе очень долго
    for i in range(min(pages, 10)):
        data = requests.get(f'https://api.hh.ru/vacancies?text={search}&per_page=100&page={i}', 
                            headers={"User-Agent":"HH-User-Agent"}).json()
        for vac in data['items']:
            salaries.append(vac['salary'])
            
    #достаем из списка всех зарплат верхние и нижние границы    
    salaries_from = []
    salaries_to = []

    for salary in salaries:
        if salary != None:
            if salary['currency'] in ['RUR', 'RUB']:
                if salary['from'] != None:
                    salaries_from.append(salary['from'])
                if salary['to'] != None:
                    salaries_to.append(salary['to'])   
    
    return (salaries_from, salaries_to)

In [50]:
def histogram(data):
    fig = px.histogram(data, nbins=30)
    
    fig.update_layout(
        title_text='распределение зарплат', # title of plot
        xaxis_title_text='зарплата', # xaxis label
        yaxis_title_text='количество вакансий', # yaxis label
        bargap=0.01, # gap between bars of adjacent location coordinates
        bargroupgap=0.01 # gap between bars of the same location coordinates
    )

    fig.update_layout(autosize=True,
                          width=800,
                          height=500,)
    
    return fig